In [ ]:
import os
from pydub import AudioSegment

def chunk_audio(input_folder, output_folder, chunk_size_samples, sr=24000, spacing=0.5, last_chunk_min_seconds=60, debug=True, errors_=None):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    errors = []
    for filename in os.listdir(input_folder):
        try:
            if not filename.endswith('.wav'):
                continue
            filepath = os.path.join(input_folder, filename)
            if errors_ is not None:
                if filepath not in errors_:
                    continue
            sound = AudioSegment.from_wav(filepath)
            sound = sound.set_frame_rate(sr)
            if debug:
                print(f'Resampled {filepath} to {sr} Hz')
            
            samples = sound.get_array_of_samples()
            n_samples = len(samples)
            n_chunks = n_samples//chunk_size_samples + (n_samples % chunk_size_samples > 0)
            start = 0
            for i in range(n_chunks):
                end = start + chunk_size_samples
                if i == n_chunks - 1 and sound.duration_seconds < last_chunk_min_seconds:
                    start = start + int( (last_chunk_min_seconds*sr - n_samples)/(n_chunks-1) )
                    end = start + chunk_size_samples
                if end > n_samples:
                    end = n_samples
                chunk = sound._spawn(samples[start:end])
                output_filename = os.path.splitext(filename)[0] + f'_chunk{i}.wav'
                output_filepath = os.path.join(output_folder, output_filename)
                chunk.export(output_filepath, format='wav')
                start = end - int(spacing*chunk_size_samples)
        except Exception as e:
            print(f'Error processing {filepath}: {e}')
            errors.append(filepath)
    return errors

In [ ]:
input_folder = 'audio'
output_folder = 'audio_chunks'
chunk_size_samples = 4000000
sr = 24000
norm = False
spacing = 0.1
debug = True

errors = chunk_audio(input_folder, output_folder, chunk_size_samples, sr=sr, norm=norm, spacing=spacing, debug=debug)